In [3]:
# Header block to include all modules that must be imported ahead of time
# Only needs to be run once per session, and each time a new module is added

import requests
from bs4 import BeautifulSoup
import json
import pandas as pd # this will need to be installed via command line first
import lxml # this will need to be installed via command line first(as well)
from babel.numbers import format_currency

In [4]:
# Method to retrieve all team names and location abbreviations 

def GetAllTeamNames():
    # Use requests to get the raw HTML response from overthecap.com
    CBSResponse = requests.get(f'https://www.cbssports.com/nfl/teams/') 
    WikiResponse = requests.get(f'https://en.wikipedia.org/wiki/Wikipedia:WikiProject_National_Football_League/National_Football_League_team_abbreviations')
    
    # If we get a normal response, proceed with data scraping
    if CBSResponse.status_code == 200 and WikiResponse.status_code == 200:
        print(f"Request succeeded with status code {CBSResponse.status_code}")
        print(f"Request succeeded with status code {WikiResponse.status_code}")

        TableTag
        
        """
        # Pandas read_html method allows the table contents to be put into a DataFrame
        TableTag = 'TableBase-table'
        DataFrames = pd.read_html(CBSResponse.content, attrs={'class': TableTag})
        
        # Select the specific parts of the dataframe we want 
        TeamsDFAFC = DataFrames[0][['East']]
        TeamsDFNFC = DataFrames[1][['East']]
        
        # Insert a new row at the top with column name
        TeamsDFAFC.insert(0, 'Conference', 'AFC')
        TeamsDFNFC.insert(0, 'Conference', 'NFC')
        
        # Generate a list of divisions in the order the appear
        DivisionsList = ['East', TeamsDFAFC.iloc[4, 1], TeamsDFAFC.iloc[9, 1], TeamsDFAFC.iloc[14, 1]]
        
        # The CBS table includes division labels as rows so they need to be removed
        TeamsDFAFC.drop(labels = [4, 9, 14], inplace = True)
        TeamsDFNFC.drop(labels = [4, 9, 14], inplace = True)
        
        # Rename the column with all the teamnames, concatinate the dataframes together
        TeamsDFAFC.rename(columns = {'East': 'TeamName'}, inplace = True)
        TeamsDFNFC.rename(columns = {'East': 'TeamName'}, inplace = True)
        TeamsDF = pd.concat([TeamsDFAFC, TeamsDFNFC], ignore_index = True)
        
        # Create list to add each division to its respective team
        DivisionsFullList = []
        for i in range(2):
            for division in DivisionsList:
                for j in range(4):
                    DivisionsFullList.append(division)
        
        # Add the new column to the teams dataframe
        TeamsDF['Division'] = DivisionsFullList
        
        # Change the order of columns to match the database schema  
        new_order = ['TeamName','Conference', 'Division']
        TeamsDF = TeamsDF.loc[:, new_order]
        print(TeamsDF)
        """
        
    # If we don't get a normal reponse, stop scraping
    else:
        print(f"Request failed with status code {CBSResponse.status_code}")
        return -1
  
GetAllTeamNames()

Request succeeded with status code 200
Request succeeded with status code 200
                 TeamName Conference Division
0           Buffalo Bills        AFC     East
1          Miami Dolphins        AFC     East
2           New York Jets        AFC     East
3    New England Patriots        AFC     East
4        Baltimore Ravens        AFC    North
5      Cincinnati Bengals        AFC    North
6        Cleveland Browns        AFC    North
7     Pittsburgh Steelers        AFC    North
8          Houston Texans        AFC    South
9      Indianapolis Colts        AFC    South
10   Jacksonville Jaguars        AFC    South
11       Tennessee Titans        AFC    South
12         Denver Broncos        AFC     West
13     Kansas City Chiefs        AFC     West
14   Los Angeles Chargers        AFC     West
15      Las Vegas Raiders        AFC     West
16         Dallas Cowboys        NFC     East
17        New York Giants        NFC     East
18    Philadelphia Eagles        NFC     East
19

/tmp/ipykernel_156/4261466018.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TeamsDFAFC.drop(labels = [4, 9, 14], inplace = True)
/tmp/ipykernel_156/4261466018.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TeamsDFNFC.drop(labels = [4, 9, 14], inplace = True)
/tmp/ipykernel_156/4261466018.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TeamsDFAFC.rename(columns = {'East': 'TeamName'}, inplace = True)
/tmp/ipykernel_156

In [9]:
# Method to get the salary cap hits for every player currently signed on an NFL team
# Creates a csv with columns player name, salary cap hit, and team location abbrviation 

def GetTeamSalaryCapHits(team):
    # Use requests to get the raw HTML response from overthecap.com
    OTCResponse = requests.get(f'https://overthecap.com/salary-cap/{team[0]}')
        
    # If we get a normal response proceed with data scraping
    if OTCResponse.status_code == 200:
        print(f"Request succeeded with status code {OTCResponse.status_code}")
        TeamNameCaptalized = (team[0].replace("-", " ")).title()
        print(f"Here are the Salary Cap hits for all players who are currently signed with the {TeamNameCaptalized}:")

        # Pandas read_html method allows the table contents to be put into a DataFrame
        TableTag = 'salary-cap-table contracted-players'
        DataFrames = pd.read_html(OTCResponse.content, attrs = {'class': TableTag})
        
        # Select the specific parts of the dataframe we want 
        SalariesDF = DataFrames[0][['Player', 'Cap Number']]
        if len(SalariesDF) >= 51:
            SalariesDF = SalariesDF.drop(51, axis=0)
        # Add team location abv to each player
        SalariesDF = SalariesDF.assign(TeamLOC = team[1])
        SalariesDF = SalariesDF.assign(TeamName = (team[0].replace("-", " ")).title())
        
        
        # Change the order of columns to make more sense semantically 
        new_order = ['Player', 'TeamLOC', 'TeamName', 'Cap Number']
        SalariesDF = SalariesDF.loc[:, new_order]
        
        # Specify filename and path
        csvPathName = f'./Salary_Lists/{TeamNameCaptalized}-Player-Salary-List.csv'
        
        # Create a .csv file with all the player names and their cap hits
        SalariesDF.to_csv(csvPathName, index = False, header = False)
    
        """
        # Read the csv back for testing purposes
        SalariesCSV = pd.read_csv(f'./Salary_Lists/{TeamNameCaptalized}-Player-Salary-List.csv', header = None)
        SalariesCSV.columns = ['Player', 'TeamLOC', 'TeamName', 'Cap Number']
        print(SalariesCSV)
        """
    # If we don't get a normal reponse, stop scraping
    else:
        print(f"Request failed with status code {OTCResponse.status_code}")
        return -1


In [11]:
# A list of links to Over The Cap Salary Cap pages for various teams
NFLTeams = [['buffalo-bills', 'BUF'], ['miami-dolphins', 'MIA'], ['new-england-patriots', 'NE'], ['new-york-jets', 'NYJ'],
            ['baltimore-ravens', 'BAL'], ['cincinnati-bengals', 'CIN'], ['cleveland-browns', 'CLE'], ['pittsburgh-steelers', 'PIT'],
            ['houston-texans', 'HOU'], ['indianapolis-colts', 'IND'], ['jacksonville-jaguars', 'JAX'], ['tennessee-titans', 'TEN'], 
            ['denver-broncos', 'DEN'], ['kansas-city-chiefs', 'KC'], ['las-vegas-raiders', 'LV'], ['los-angeles-chargers', 'LAC'],
            ['dallas-cowboys', 'DAL'], ['new-york-giants', 'NYG'], ['philadelphia-eagles', 'PHI'], ['washington-commanders', 'WSH'],
            ['chicago-bears', 'CHI'], ['detroit-lions', 'DET'], ['green-bay-packers', 'GB'], ['minnesota-vikings', 'MIN'],
            ['atlanta-falcons', 'ATL'], ['carolina-panthers', 'CAR'], ['new-orleans-saints', 'NO'], ['tampa-bay-buccaneers', 'TB'],
            ['arizona-cardinals', 'ARI'], ['los-angeles-rams', 'LAR'], ['san-francisco-49ers', 'SF'], ['seattle-seahawks', 'SEA']
           ]

# Call the method defined above to get all the player data
def GetMultipleTeamSalaryCapHits(TeamList):
    for team in TeamList:
        GetTeamSalaryCapHits(team)

# GetMultipleTeamSalaryCapHits(NFLTeams)
GetTeamSalaryCapHits(['san-francisco-49ers', 'SF'])

Request succeeded with status code 200
Here are the Salary Cap hits for all players who are currently signed with the San Francisco 49Ers:
                 Player TeamLOC             TeamName   Cap Number
0        Trent Williams      SF  San Francisco 49Ers  $31,568,542
1          Deebo Samuel      SF  San Francisco 49Ers  $28,633,765
2         Arik Armstead      SF  San Francisco 49Ers  $28,353,236
3           Fred Warner      SF  San Francisco 49Ers  $24,469,000
4         George Kittle      SF  San Francisco 49Ers  $21,956,575
5       Charvarius Ward      SF  San Francisco 49Ers  $18,401,000
6        Javon Hargrave      SF  San Francisco 49Ers  $15,430,882
7             Nick Bosa      SF  San Francisco 49Ers  $14,671,000
8   Christian McCaffrey      SF  San Francisco 49Ers  $14,144,000
9         Brandon Aiyuk      SF  San Francisco 49Ers  $14,124,000
10         Dre Greenlaw      SF  San Francisco 49Ers   $9,602,970
11        Kyle Juszczyk      SF  San Francisco 49Ers   $7,588,750
12 